<a href="https://colab.research.google.com/github/amy0621/LLM/blob/main/%E9%96%8B%E6%BA%90test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/grasses/PoisonPrompt.git
%cd PoisonPrompt


fatal: destination path 'PoisonPrompt' already exists and is not an empty directory.
/content/PoisonPrompt


In [2]:
# 先卸掉可能已安裝的新版
!pip uninstall -y peft transformers

# 安裝相容組合
!pip install transformers==4.37.2 peft==0.10.0


Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
Found existing installation: transformers 4.30.1
Uninstalling transformers-4.30.1:
  Successfully uninstalled transformers-4.30.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 whi

In [3]:
!pip install -r requirements.txt

  Using cached transformers-4.30.1-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.30.1-py3-none-any.whl (7.2 MB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.1 which is incompatible.


In [4]:
%env model_name=roberta-large
%env TASK_NAME=glue
%env DATASET_NAME=sst2
%env CUDA_VISIBLE_DEVICES=0
%env bs=24
%env lr=3e-4
%env dropout=0.1
%env psl=32
%env epoch=4
# 把下方的 label/token id 填上 README 提供的那些清單
%env label2ids='{"0":[31321,34858,23584,32650,3007,21223,38323,34771,37649,35907,45103,31846,31790,13689,27112,30603,36100,14260,38821,16861],"1":[27658,30560,40578,22653,22610,26652,18503,11577,20590,18910,30981,23812,41106,10874,44249,16044,7809,11653,15603,8520]}'
%env label2bids='{"0":[2,1437,22,0,36,50141,10,364,5,1009,385,2156,784,8,579,19246,910,4,4832,6],"1":[2,1437,22,0,36,50141,10,364,5,1009,385,2156,784,8,579,19246,910,4,4832,6]}'



env: model_name=roberta-large
env: TASK_NAME=glue
env: DATASET_NAME=sst2
env: CUDA_VISIBLE_DEVICES=0
env: bs=24
env: lr=3e-4
env: dropout=0.1
env: psl=32
env: epoch=4
env: label2ids='{"0":[31321,34858,23584,32650,3007,21223,38323,34771,37649,35907,45103,31846,31790,13689,27112,30603,36100,14260,38821,16861],"1":[27658,30560,40578,22653,22610,26652,18503,11577,20590,18910,30981,23812,41106,10874,44249,16044,7809,11653,15603,8520]}'
env: label2bids='{"0":[2,1437,22,0,36,50141,10,364,5,1009,385,2156,784,8,579,19246,910,4,4832,6],"1":[2,1437,22,0,36,50141,10,364,5,1009,385,2156,784,8,579,19246,910,4,4832,6]}'


In [8]:
%%bash
export model_name=roberta-large
export label2ids='{"0":[31321,34858,...],"1":[27658,30560,...]}'
export label2bids='{"0":[2,1437,...],"1":[2,1437,...]}'
export TASK_NAME=glue
export DATASET_NAME=sst2
export CUDA_VISIBLE_DEVICES=0
export bs=24
export lr=3e-4
export dropout=0.1
export psl=32
export epoch=4

python step1_attack.py \
  --model_name_or_path ${model_name} \
  --task_name $TASK_NAME \
  --dataset_name $DATASET_NAME \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size $bs \
  --learning_rate $lr \
  --num_train_epochs $epoch \
  --pre_seq_len $psl \
  --output_dir checkpoints/$DATASET_NAME-${model_name}/ \
  --overwrite_output_dir \
  --hidden_dropout_prob $dropout \
  --seed 2233 \
  --save_strategy epoch \
  --evaluation_strategy epoch \
  --prompt \
  --trigger_num 5 \
  --trigger_cand_num 40 \
  --backdoor targeted \
  --backdoor_steps 500 \
  --warm_steps 500 \
  --clean_labels $label2ids \
  --target_labels $label2bids


2025-07-27 07:55:37.626433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753602937.672475    5391 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753602937.685390    5391 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Traceback (most recent call last):
  File "/content/PoisonPrompt/step1_attack.py", line 81, in <module>
    args = get_args()
           ^^^^^^^^^^
  File "/content/PoisonPrompt/arguments.py", line 323, in get_args
    for k, v in json.lo

CalledProcessError: Command 'b'export model_name=roberta-large\nexport label2ids=\'{"0":[31321,34858,...],"1":[27658,30560,...]}\'\nexport label2bids=\'{"0":[2,1437,...],"1":[2,1437,...]}\'\nexport TASK_NAME=glue\nexport DATASET_NAME=sst2\nexport CUDA_VISIBLE_DEVICES=0\nexport bs=24\nexport lr=3e-4\nexport dropout=0.1\nexport psl=32\nexport epoch=4\n\npython step1_attack.py \\\n  --model_name_or_path ${model_name} \\\n  --task_name $TASK_NAME \\\n  --dataset_name $DATASET_NAME \\\n  --do_train \\\n  --do_eval \\\n  --max_seq_length 128 \\\n  --per_device_train_batch_size $bs \\\n  --learning_rate $lr \\\n  --num_train_epochs $epoch \\\n  --pre_seq_len $psl \\\n  --output_dir checkpoints/$DATASET_NAME-${model_name}/ \\\n  --overwrite_output_dir \\\n  --hidden_dropout_prob $dropout \\\n  --seed 2233 \\\n  --save_strategy epoch \\\n  --evaluation_strategy epoch \\\n  --prompt \\\n  --trigger_num 5 \\\n  --trigger_cand_num 40 \\\n  --backdoor targeted \\\n  --backdoor_steps 500 \\\n  --war